In [ ]:
from avapi.rccars import RcCarsScenesManager

scene_manager = RcCarsScenesManager(data_dir="/data/shared/rccars")
print(f"Available scenes: {scene_manager.scenes}")

mmdet_root = "../third_party/avstack-core/third_party/mmdetection"
mmseg_root = "../third_party/avstack-core/third_party/mmsegmentation"

## Dataset visualization

In [ ]:
from avapi.visualize.snapshot import show_image_with_boxes


# get scene manager
scene_dataset = scene_manager.get_scene_dataset_by_name("oneclass-detection-2025", "val")

# get image and boxes
idx_frame = 1
frame = scene_dataset.get_frames(idx_frame)[idx_frame]
img = scene_dataset.get_image(frame=frame)
boxes = scene_dataset.get_boxes(frame=frame, sensor="camera")
show_image_with_boxes(img=img, boxes=boxes, inline=True, show=True)

## Bounding box detection algorithm

In [ ]:
import numpy as np
from avstack.modules.assignment import build_A_from_iou, gnn_single_frame_assign
from avstack.modules.perception.object2dfv import MMDetObjectDetector2D


def compute_metrics(objs_det, objs_tru, do_print=False):
    # compute metrics
    # -- perform assignment
    A_assign = build_A_from_iou(objs_det, objs_tru)
    assigns = gnn_single_frame_assign(A_assign)

    # -- report fp/rn rate
    n_fp = len(assigns.unassigned_rows)
    n_fn = len(assigns.unassigned_cols)
    if do_print:
        print(f"{assigns.nrow} detected\n---------------------")
        print(f"{assigns.nrow - n_fp} assignment")
        if assigns.nrow > 0:
            print(f"{n_fp} false positives ({100*n_fp/assigns.nrow:.2f}% of detections)")
        else:
            print("No detections, so no FP rate")
        print(f"{n_fn} false negatives ({100*n_fn/assigns.ncol:.2f}% of truths)")

    # -- report average IoU of matches
    ious = [objs_det[a[0]].IoU(objs_tru[a[1]]) for a in assigns.assignment_tuples]
    if do_print:
        print(f"Matches had IoU of: [{', '.join(map(str, ious))}] each")
        print(f"Mean IoU: {np.mean(ious):.2f}")

    return assigns, n_fp, n_fn, ious



def load_model(epoch, threshold: float):
    # set up this model
    detector = MMDetObjectDetector2D(
        model="rtmdet",
        dataset="rccars-oneclass",
        deploy=False,
        threshold=threshold,
        gpu=0,
        epoch=epoch,
    )
    return detector

In [ ]:
# load the model
epoch = "latest"  # sweep this from 1 - 100
threshold = 0.7  # between [0, 1]. sweep this from 0.1 to 0.9
detector = load_model(epoch=epoch, threshold=threshold)

In [ ]:
# test the model on some examples from train and val
idx_frame = 1
frame = scene_dataset.get_frames(idx_frame)[idx_frame]
img = scene_dataset.get_image(frame=frame)
objs_det = detector(img)

# metrics
objs_tru = scene_dataset.get_boxes(frame=frame, sensor="camera")
assigns, n_fp, n_fn, ious = compute_metrics(objs_det, objs_tru)

In [ ]:
# visualize the detected
print("DETECTED")
det_colors = [
    "green" if assigns.has_assign(row=i) else "red"
    for i in range(len(objs_det))
]
show_image_with_boxes(
    img=img,
    boxes=objs_det,
    box_colors=det_colors,
    inline=True,
    show=True,
)

# visualize the truth
print("GROUND TRUTH")
show_image_with_boxes(
    img=img,
    boxes=objs_tru,
    box_colors="white",
    inline=True,
    show=True,
)